<a href="https://colab.research.google.com/github/756616071/AiTestPublic/blob/main/nb/dsT1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [1]:
%%capture
# Normally using pip install unsloth is enough

# Temporarily as of Jan 31st 2025, Colab has some issues with Pytorch
# Using pip install unsloth will take 3 minutes, whilst the below takes <1 minute:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

### Unsloth

In [2]:
print('开始调试,引入依赖...')
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
max_seq_length = 2048 # Supports RoPE Scaling interally, so choose any!
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
    "unsloth/Phi-3-mini-4k-instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth
#模型载入
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,#步长
    dtype = None,# 设置数据类型
    load_in_4bit = True,# 启用4bit量化加载
)
print('模型载入完成')

train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", 'zh', split = "train[0:500]", trust_remote_code=True)
print(dataset.column_names)
dataset = dataset.map(formatting_prompts_func, batched = True)
dataset["text"][0]
print('开始包装')
# Do model patching and add fast LoRA weights
# 基础模型和peft_config包装起来
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,# LoRA的秩(rank)值，决定了低秩矩阵的维度，较大的r值(如16)可以提供更强的模型表达能力，但会增加参数量和计算开销，较小的r值(如4或8)则会减少参数量，但可能影响模型性能，通常在4-16之间选择,需要在性能和效率之间权衡
    target_modules = [ #指定需要应用LoRA微调的模块列表，q_proj, k_proj, v_proj: 注意力机制中的查询、键、值投影层
                    "q_proj",
                      "k_proj",
                      "v_proj",
                      "o_proj",#注意力输出投影层
                      "gate_proj", "up_proj", "down_proj",],
        lora_alpha=16, #缩放参数，用于控制LoRA更新的强度，通常设置为与r相同的值，较大的alpha会增加LoRA的影响力，较小的alpha则会减弱LoRA的影响
        lora_dropout=0, #LoRA层的dropout率，0表示不使用dropout，增加dropout可以帮助防止过拟合，但可能影响训练稳定性，在微调时通常设为0或很小的值
        bias="none", #是否微调偏置项，"none"表示不微调偏置参数，也可以设置为"all"或"lora_only"来微调不同范围的偏置
        use_gradient_checkpointing="unsloth",  # 梯度检查点策略，"unsloth"是一种优化的检查点策略，适用于长上下文可以显著减少显存使用，但会略微增加计算时间对处理长文本特别有用
        random_state=3407, #随机数种子，控制初始化的随机性，固定种子可以确保实验可重复性
        use_rslora=False, #是否使用RSLoRA(Rank-Stabilized LoRA) False表示使用标准LoRARSLoRA是一种改进的LoRA变体，可以提供更稳定的训练
        loftq_config=None, #LoftQ配置None表示不使用LoftQ量化LoftQ是一种用于模型量化的技术，可以减少模型大小
        # max_seq_length = max_seq_length,
)
print ("基础模型和peft_config包装起来-->完成")
print ("开始微调")
#训练工具 微调参数设置
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",# question  text
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
         # 批处理相关
        per_device_train_batch_size = 2,# 每个设备（GPU）的训练批次大小
        gradient_accumulation_steps = 4,# 梯度累积步数，用于模拟更大的批次大小
          # 训练步数和预热
        warmup_steps = 5,# 学习率预热步数，逐步增加学习率
        max_steps = 60,# 最大训练步数  要设置到恰到好处  就像人锻炼一样  跑多少米停下来
        learning_rate=2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,# 每10步记录一次日志
        weight_decay=0.01,# 权重衰减率，用于防止过拟合
        lr_scheduler_type="linear",# 学习率调度器类型，使用线性衰减
        output_dir = "outputs",# 模型和检查点的输出目录
        optim = "adamw_8bit",# 使用8位精度的 AdamW 优化器
        seed = 3407,# 随机种子，确保实验可重复性
    ),
)
# 开始微调
trainer.train()
print ("微调完成")

开始调试,引入依赖...
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: w756616071 (w756616071-czzx) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
1,2.274600
2,2.348500
3,2.289000
4,2.139000
5,2.032100
6,1.987100
7,2.117700
8,1.902600
9,1.966900
10,1.858100


微调完成


In [ ]:
# 测试问答
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>{}"""
question = "一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？"


FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])

